In [1]:
import pandas as pd
import numpy as np
from typing import List, Dict, Any
from fastapi import FastAPI, Request, HTTPException
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates
from fastapi.responses import FileResponse, HTMLResponse
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer
import uvicorn
import os

### Consulta 5

In [8]:
def sentiment_analysis(year:int) -> dict:
    # Cargar el DataFrame desde el archivo parquet
    df = pd.read_parquet('venv/data/users_reviews_etl_comprimido.parquet')

    # Filtrar las filas del DataFrame para el año proporcionado
    df_year = df[df['date'] == int(year)]

    # Contar los valores en la columna 'sentiment_analysis'
    count_negative = (df_year['sentiment_analysis'] == 0).sum()
    count_neutral = (df_year['sentiment_analysis'] == 1).sum()
    count_positive = (df_year['sentiment_analysis'] == 2).sum()

    # Crear un diccionario con los resultados
    result_mapped = {
        'Negative': count_negative,
        'Neutral': count_neutral,
        'Positive': count_positive
    }

    # Eliminar las claves que tengan un valor de 0
    #result_mapped = {key: value for key, value in result_mapped.items() if value != 0}

    return result_mapped

In [9]:
# Ejemplo de uso de la función
year = 2011
analysis_result = sentiment_analysis(year)
print(analysis_result)

{'Negative': 114, 'Neutral': 72, 'Positive': 346}


### Consulta 3

In [26]:

def UsersRecommend(year: int) -> List[Dict[str, Any]]:
    # Cargar los DataFrames desde los archivos parquet
    df_reviews = pd.read_parquet('venv/data/users_reviews_etl_comprimido.parquet')
    df_items = pd.read_parquet('venv/data/users_items_etl_comprimido.parquet')
    
    # Filtrar por año y recommend=True
    filtered_reviews = df_reviews[(df_reviews['date'] == int(year)) & (df_reviews['recommend'] == True)]
    
    # Obtener los tres primeros item_id con el mayor número de sentiment_analysis igual a 1 o 2
    top_items = filtered_reviews.groupby('item_id').sum().sort_values(by='sentiment_analysis', ascending=False).head(3)
    
    # Obtener los nombres de los juegos correspondientes a los item_id encontrados
    recommendations = []
    for idx, row in top_items.iterrows():
        # Verificar si el item_id existe en df_items
        if idx in df_items['item_id'].values:
            item_name = df_items.loc[df_items['item_id'] == idx, 'item_name'].iloc[0]
            recommendations.append({"Puesto " + str(len(recommendations) + 1): item_name})
        else:
            recommendations.append({"Puesto " + str(len(recommendations) + 1): idx})
    
    return recommendations

In [34]:
# Ejemplo de uso:
year = 2009  # Año ingresado por el usuario
recommendations = UsersRecommend(year)
print(recommendations)

[]


### Consulta 4

In [35]:
def UsersNotRecommend(year:int) -> List[Dict[str, Any]]:
    # Cargar los DataFrames desde los archivos parquet
    df_reviews = pd.read_parquet('venv/data/users_reviews_etl_comprimido.parquet')
    df_items = pd.read_parquet('venv/data/users_items_etl_comprimido.parquet')
    
    # Filtrar por año y recommend=False
    filtered_reviews = df_reviews[(df_reviews['date'] == int(year)) & (df_reviews['recommend'] == False)]
    
    # Obtener los tres primeros item_id con el mayor número de sentiment_analysis igual a 0
    top_items = filtered_reviews.groupby('item_id').sum().sort_values(by='sentiment_analysis', ascending=False).head(3)
    
    # Obtener los nombres de los juegos correspondientes a los item_id encontrados
    recommendations = []
    for idx, row in top_items.iterrows():
        # Verificar si el item_id existe en df_items
        if idx in df_items['item_id'].values:
            item_name = df_items.loc[df_items['item_id'] == idx, 'item_name'].iloc[0]
            recommendations.append({"Puesto " + str(len(recommendations) + 1): item_name})
        else:
            recommendations.append({"Puesto " + str(len(recommendations) + 1): idx})
    
    return recommendations

In [39]:
# Ejemplo de uso:
year = 2012  # Año ingresado por el usuario
recommendations = UsersNotRecommend(year)
print(recommendations)

[{'Puesto 1': 'Call Of Duty Modern Warfare 3'}, {'Puesto 2': 'Aliens Vs Predator'}, {'Puesto 3': 'Jagged Alliance Back In Action'}]


### Consulta 1

In [7]:

def UsersRecommend(genre: str) -> Dict[str, int]:
    # Cargar los DataFrames
    df_games =pd.read_parquet('venv/data/steam_games_etl_comprimido.parquet')
    df_items = pd.read_parquet('venv/data/users_items_etl_comprimido.parquet')
    
    # Filtrar por género
    genre_games = df_games[df_games['genres'].str.contains(genre, case=False)]
    
    # Obtener los IDs de los videojuegos del género especificado
    genre_game_ids = genre_games['id_game'].unique()
    
    # Filtrar los items por los IDs de los videojuegos del género
    genre_items = df_items[df_items['item_id'].isin(genre_game_ids)]
    
    # Sumar el tiempo de juego en horas para cada videojuego
    genre_items.loc[:, 'playtime_hours'] = genre_items['playtime_forever'] / 60
    
    # Unir los DataFrames para obtener los años de lanzamiento de los videojuegos del género
    merged_df = pd.merge(genre_items, df_games[['id_game', 'year_release']], left_on='item_id', right_on='id_game')
    
    # Agrupar por año de lanzamiento y sumar las horas jugadas
    year_playtime = merged_df.groupby('year_release')['playtime_hours'].sum()
    
    # Encontrar el año con más horas jugadas
    max_playtime_year = year_playtime.idxmax()
    
    # Construir el resultado final
    result = {f"Año de lanzamiento con más horas jugadas para Género {genre}": max_playtime_year}
    
    return result

In [8]:
# Ejemplo de uso
genre = "Sports"  # Género ingresado por el usuario
recommendation = UsersRecommend(genre)
print(recommendation)

{'Año de lanzamiento con más horas jugadas para Género Sports': 2015}


C:\Users\jhcat\AppData\Local\Temp\ipykernel_10144\573753200.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genre_items.loc[:, 'playtime_hours'] = genre_items['playtime_forever'] / 60


# Consulta 2

In [13]:
def UserForGenre(genre: str) -> List[Dict[str, Any]]:
    # Cargar los DataFrames
    df_games = pd.read_parquet('venv/data/steam_games_etl_comprimido.parquet')
    df_items = pd.read_parquet('venv/data/users_items_etl_comprimido.parquet')
    
    # Filtrar por género
    genre_games = df_games[df_games['genres'].str.contains(genre, case=False)]
    
    # Obtener los IDs de los videojuegos del género especificado
    genre_game_ids = genre_games['id_game'].unique()
    
    # Filtrar los items por los IDs de los videojuegos del género
    genre_items = df_items[df_items['item_id'].isin(genre_game_ids)]
    
    # Sumar el tiempo de juego en horas para cada videojuego y usuario
    genre_items.loc[:, 'playtime_hours'] = genre_items['playtime_forever'] / 60
    
    # Agrupar por usuario y sumar las horas jugadas para cada uno
    user_playtime = genre_items.groupby('user_id').agg({'playtime_hours': 'sum'})
    
    # Encontrar al usuario con más horas jugadas
    user_with_most_playtime = user_playtime.idxmax()[0]
    
    # Obtener los juegos donde el usuario con más horas jugadas jugó
    games_played_by_user = genre_items[genre_items['user_id'] == user_with_most_playtime]['item_name'].unique()
    
    # Volver al DataFrame df_games para cruzar con las horas jugadas por año
    merged_df = pd.merge(df_games, genre_items, left_on='app_name', right_on='item_name')
    
    # Agrupar por año y sumar las horas jugadas para el usuario y género especificados
    user_genre_year_playtime = merged_df.groupby(['user_id', 'year_release']).agg({'playtime_hours': 'sum'})
    
    # Obtener las horas jugadas por año para el usuario con más horas jugadas
    if user_with_most_playtime in user_genre_year_playtime.index:
        user_most_playtime_year = user_genre_year_playtime.loc[user_with_most_playtime].reset_index()
    else:
        user_most_playtime_year = pd.DataFrame(columns=['year_release', 'playtime_hours'])
    
    # Construir el resultado final
    result = {
        "Usuario con más horas jugadas para Género {}: ".format(genre): user_with_most_playtime,
        "Horas jugadas": [
            {"Año": int(row['year_release']), "Horas": row['playtime_hours']} for index, row in user_most_playtime_year.iterrows()
        ]
    }
    
    return result

In [17]:
# Ejemplo de uso
genre = "Casual"  # Género ingresado por el usuario
recommendation = UserForGenre(genre)
print(recommendation)

C:\Users\jhcat\AppData\Local\Temp\ipykernel_10144\1694948807.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genre_items.loc[:, 'playtime_hours'] = genre_items['playtime_forever'] / 60
C:\Users\jhcat\AppData\Local\Temp\ipykernel_10144\1694948807.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  user_with_most_playtime = user_playtime.idxmax()[0]


{'Usuario con más horas jugadas para Género Casual: ': 'Overwhelmer', 'Horas jugadas': [{'Año': 2012, 'Horas': 1.0833333333333333}, {'Año': 2015, 'Horas': 3982.8}]}
